In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import pickle

In [2]:
course_chapter_items_df = pd.read_csv('./data/course_chapter_items.csv')
#chapter_items_seq = pd.read_csv('./data/course_chapter_items_sequence.csv')
course_df = pd.read_csv('./data/courses.csv')
users_df = pd.read_csv('./data/users.csv')
subgroups_df = pd.read_csv('./data/subgroups.csv')

train_group_df = pd.read_csv('./data/train_group.csv')
test_seen_group_df = pd.read_csv('./data/test_seen_group.csv')
val_seen_group_df = pd.read_csv('./data/val_seen_group.csv')
test_unseen_group_df = pd.read_csv('./data/test_unseen_group.csv')
val_unseen_group_df = pd.read_csv('./data/val_unseen_group.csv')

train_df = pd.read_csv('./data/train.csv')
test_seen_df = pd.read_csv('./data/test_seen.csv')
val_seen_df = pd.read_csv('./data/val_seen.csv')
test_unseen_df = pd.read_csv('./data/test_unseen.csv')
val_unseen_df = pd.read_csv('./data/val_unseen.csv')

In [3]:
course_df.head(2)

,course_id,course_name,course_price,teacher_id,teacher_intro,groups,sub_groups,groups+subgroups,topics,course_published_at_local,description,will_learn,required_tools,recommended_background,target_group
0,61888e868f154b000781b45a,少女人妻華麗變身：七大妝容七彩的夢幻樂園,1800,61888e7bb7fe1c0006850eff,在美妝 KOL 圈裡屬個人風格強烈的 Alice，在清新與叛逆風格間遊刃有餘，其幽默的美妝影...,生活品味,"更多生活品味,護膚保養與化妝","生活品味_更多生活品味,生活品味_護膚保養與化妝","更多生活品味,護膚保養與化妝",NaN,"<img src=""https://images.api.hahow.in/images/6...",不再害怕各種顏色的彩妝，可以更隨心搭配各種繽紛的顏色。\n,所需工具為：視課程實際會用到的彩妝用品,只要你有一顆愛化妝、想變漂亮的心皆可以參加。\n\n⚠️ 雖然課程當中會帶到相關彩妝技巧，不...,熱愛彩妝的人
1,54d5a117065a7e0e00725ac0,幾何圖形分割 X 色塊組合,100,54d5a079065a7e0e00725abe,從學生時代開始摸索photoshop等軟體，自以為有些天賦但後來發現其實沒有。出社會後從事美...,"藝術,設計","平面設計,繪畫與插畫","設計_平面設計,藝術_繪畫與插畫","Illustrator/以拉,配色技巧",42090.98472,"<img src=""https://storage.googleapis.com/hahow...",可以將任何區塊分割成自己想要的幾何圖形、快速的上色，並且能夠應用在許多地方,"Adobe Illustrator（必備）, camera",知道如何使用Illustrator的基本工具列,每一位興趣的人都能學得來，非常容易的小技巧


# 課程 Vectorize

In [4]:
id2course_mapping = course_df["course_id"].to_dict()
course2id_mapping = {v : k for k, v in id2course_mapping.items()}
len(course2id_mapping)

id2course_mapping = course_df["course_id"].to_dict()
course2id_mapping = {v : k for k, v in id2course_mapping.items()}
print(len(course2id_mapping))

id2user_mapping = users_df["user_id"].to_dict()
user2id_mapping = {v : k for k, v in id2user_mapping.items()}
len(id2user_mapping)


728


130566

In [5]:
def clean_data(x):
    return x.replace(" ", "")

fillna=course_df.fillna('')
features= ['course_name', 'teacher_intro', 'groups', 'sub_groups', 'topics', 'will_learn', 'recommended_background', 'target_group', 'required_tools']
fillna=fillna[features]
for feature in features:
    fillna[feature] = fillna[feature].apply(clean_data)
    
fillna.head(5)

,course_name,teacher_intro,groups,sub_groups,topics,will_learn,recommended_background,target_group,required_tools
0,少女人妻華麗變身：七大妝容七彩的夢幻樂園,在美妝KOL圈裡屬個人風格強烈的Alice，在清新與叛逆風格間遊刃有餘，其幽默的美妝影片手法...,生活品味,"更多生活品味,護膚保養與化妝","更多生活品味,護膚保養與化妝",不再害怕各種顏色的彩妝，可以更隨心搭配各種繽紛的顏色。\n,只要你有一顆愛化妝、想變漂亮的心皆可以參加。\n\n⚠️雖然課程當中會帶到相關彩妝技巧，不過...,熱愛彩妝的人,所需工具為：視課程實際會用到的彩妝用品
1,幾何圖形分割X色塊組合,從學生時代開始摸索photoshop等軟體，自以為有些天賦但後來發現其實沒有。出社會後從事美...,"藝術,設計","平面設計,繪畫與插畫","Illustrator/以拉,配色技巧",可以將任何區塊分割成自己想要的幾何圖形、快速的上色，並且能夠應用在許多地方,知道如何使用Illustrator的基本工具列,每一位興趣的人都能學得來，非常容易的小技巧,"AdobeIllustrator（必備）,camera"
2,數位拼貼的手感,自由工作者，致力於品牌視覺規劃，包含識別系統設計、平面設計、包裝設計、網頁、插畫製作等，偶爾...,"藝術,設計","平面設計,電腦繪圖",photoshop,藉由這堂課，可以學習到如何利用身邊的物件自製素材，了解去背、Photoshop的基本操作，並...,,繪圖軟體初學者；喜歡隨性創作；常天馬行空；喜歡剪剪貼貼或收集東西，卻不知該如何收藏。,軟體：AdobePhotoshop、AdobeIllustrator；\n設備：電腦、相機、...
3,Line的貼圖自己動手做！,我是Danny，現是一名自由接案的插畫師，主要是做插畫設計、人物設計、AppUI設計，201...,"藝術,設計","平面設計,應用設計,電腦繪圖","LINE貼圖設計,photoshop",如果上完課之後好好練習(不懂的地方隨時提問)，就可以畫出你在一般市面上看的到的line貼圖(...,只要手可以拿得起鉛筆和握滑鼠就可以,喜歡畫畫，但是沒有學過電腦繪圖的人、想學基本的電腦繪圖但不知如何入手的人，從10歲-100歲...,Photoshop(可至官網下載試用版)
4,為申請學校或工作寫好英文自傳,在北美長大，回到南台灣尋根後，現落腳於南加州的庄腳嬰仔。在鑽研細胞奈米科學與皮膚再生醫學的過...,"職場技能,語言","求職,英文","商用英文,履歷撰寫","Apersonalstatementthatyoucanbeproudof,andarewa...",HowtouseGoogleDoc.BasicEnglishlisteningcompreh...,Peopleapplyingforcollege/post-graduatestudieso...,GoogleDocwillbeusedtosubmit/shareyourassignmen...


In [6]:
def create_combination(x):
    course_name = x['course_name']
    teacher_intro = x['teacher_intro']
    will_learn = x['will_learn']
    recommended_background = x['recommended_background']
    target_group = x['target_group']
    required_tools = x['required_tools']
    text = course_name + ' ' + teacher_intro + ' ' + x['groups'] + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + recommended_background + ' ' + target_group
    return text

fillna['combination'] = fillna.apply(create_combination, axis=1)
fillna['combination'].head(5)

0    少女人妻華麗變身：七大妝容七彩的夢幻樂園 在美妝KOL圈裡屬個人風格強烈的Alice，在清新...
1    幾何圖形分割X色塊組合 從學生時代開始摸索photoshop等軟體，自以為有些天賦但後來發現...
2    數位拼貼的手感 自由工作者，致力於品牌視覺規劃，包含識別系統設計、平面設計、包裝設計、網頁、...
3    Line的貼圖自己動手做！ 我是Danny，現是一名自由接案的插畫師，主要是做插畫設計、人物...
4    為申請學校或工作寫好英文自傳 在北美長大，回到南台灣尋根後，現落腳於南加州的庄腳嬰仔。在鑽研...
Name: combination, dtype: object

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import datetime
from scipy.sparse import csr_matrix

In [8]:
tfidf = TfidfVectorizer(stop_words= None)
#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(fillna['combination'])

#print(tfidf.vocabulary_)
tfidf_matrix_nd = tfidf_matrix.toarray()
np.save('tfidf_embed/item_embed_.npy', tfidf_matrix_nd)

In [9]:
print(len(tfidf.get_feature_names_out()))
list(tfidf.vocabulary_.items())[:5]

23722


[('少女人妻華麗變身', 9981),
 ('七大妝容七彩的夢幻樂園', 2036),
 ('在美妝kol圈裡屬個人風格強烈的alice', 7722),
 ('在清新與叛逆風格間遊刃有餘', 7693),
 ('其幽默的美妝影片手法引起許多粉絲的共鳴', 5165)]

In [10]:
from scipy.sparse import csr_matrix
item_embed_FM = np.load('tfidf_embed/item_embed_.npy')
item_embed_FM = item_embed_FM[:728]
print(tfidf_matrix.shape)
print(item_embed_FM.shape)

(728, 23722)
(728, 23722)


In [11]:
concatenate_item_embed_nd = np.concatenate([tfidf_matrix_nd, item_embed_FM/50], axis = 1)
print(concatenate_item_embed_nd.shape)
concatenate_item_embed = csr_matrix(concatenate_item_embed_nd)
# print(concatenate_item_embed[0])

(728, 47444)


In [12]:
# Compute the cosine similarity matrix
print(concatenate_item_embed_nd.shape)
cosine_sim2 = linear_kernel(concatenate_item_embed, concatenate_item_embed)
#Output the shape of tfidf_matrix
cosine_sim2.shape
# count = CountVectorizer(stop_words='english')
# count_matrix = count.fit_transform(fillna['combination'])

# cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

(728, 47444)


(728, 728)

In [13]:
import pickle
tfidf_save = {"TfidfVectorizer": tfidf, "Tfidf_matrix": tfidf_matrix, "lightFM_embeding_matrix": item_embed_FM, "concatenate_matrix": concatenate_item_embed}
with open('subgroup_content_embed/all.pickle', 'wb') as f:
    pickle.dump(tfidf_save, f)

# 根據課程相似度 Predict Subgroups

In [64]:
subgroups_df = pd.read_csv('./data/subgroups.csv')
subgroup_num = len(subgroups_df)

subgroups2idx = {}
for (id, name) in zip(subgroups_df["subgroup_id"], subgroups_df["subgroup_name"]):
    subgroups2idx.update({name: id})
idx2subgroups = {v : k for k, v in subgroups2idx.items()}

course2subgroups = {}
for (course_id, sub_groups) in zip(course_df["course_id"], course_df["sub_groups"]):
    if pd.isnull(sub_groups):
        course2subgroups.update({course_id: [0]})
    else:
        course2subgroups.update({course_id: [ subgroups2idx[sub_group] for sub_group in sub_groups.split(',')]})

list(course2subgroups.items())[:5]

[('61888e868f154b000781b45a', [1, 2]),
 ('54d5a117065a7e0e00725ac0', [3, 4]),
 ('54d5d9952246e60a009ec571', [3, 5]),
 ('54d7148a2246e60a009ec588', [3, 6, 5]),
 ('5513e92b38239d10005778e1', [7, 8])]

In [65]:
def get_recommendations_new(haved_courses_list, cosine_sim, top = 50):
    cosine_sim_sum = [ [i, 0] for i in range(len(course2id_mapping))]
    haved_courses_index_list = [ course2id_mapping[course_id] for course_id in haved_courses_list]
    for idx in haved_courses_index_list:
        # Get the pairwsie similarity scores of all courses with that course
        sim_scores = list(enumerate(cosine_sim[idx]))
        for i in range(len(sim_scores)):
            cosine_sim_sum[i][1] += sim_scores[i][1]

    # Sort the similarity scores
    sim_scores = sorted(cosine_sim_sum, key=lambda x: x[1], reverse=True)

    recommend_subgroups = []
    # Get the scores of the top most course not buy
    for i in range(len(sim_scores)):
        if len(recommend_subgroups) < top:
            if sim_scores[i][0] not in haved_courses_index_list:
                course_id = course_df['course_id'][sim_scores[i][0]]
                for subgroup in course2subgroups[course_id]:
                    if subgroup not in recommend_subgroups:
                        recommend_subgroups.append(subgroup)

    return recommend_subgroups

test = ['5513e92b38239d10005778e1']
print(course_df['course_name'].iloc[[course2id_mapping[course] for course in test]].tolist())  
test_predict = get_recommendations_new(test, cosine_sim2, 10)
[ idx2subgroups[subgroup_id] for subgroup_id in test_predict ]

['為申請學校或工作寫好英文自傳']


['職場溝通',
 '英文',
 '求職',
 '更多生活品味',
 '親子教育',
 '個人品牌經營',
 '更多職場技能',
 '網站架設',
 '平面設計',
 '電腦繪圖']

In [66]:
seen_predict_df = pd.read_csv('./data/test_seen_group.csv')
predict_user = seen_predict_df["user_id"].to_list()
print(len(predict_user))


seen_user_haved_purchased_course = {}
for seen_user_id, course_ids in zip(train_df["user_id"], train_df["course_id"]):
    course_id_list = course_ids.split(' ')
    seen_user_haved_purchased_course[seen_user_id] = seen_user_haved_purchased_course.setdefault(seen_user_id, []) + course_id_list
for seen_user_id, course_ids in zip(val_seen_df["user_id"], val_seen_df["course_id"]):
    course_id_list = course_ids.split(' ')
    seen_user_haved_purchased_course[seen_user_id] = seen_user_haved_purchased_course.setdefault(seen_user_id, []) + course_id_list
print(len(seen_user_haved_purchased_course))

7205
59737


# Predict and Save

In [67]:
import csv

predict_users = seen_predict_df["user_id"].to_list()
with open("subgroup_predict.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["user_id", "subgroup"])
    for user_id in predict_users:
        recommend_subgroups_str = [ str(x) for x in get_recommendations_new(seen_user_haved_purchased_course[user_id], cosine_sim2, top = 50)]
        recommend = " ".join(recommend_subgroups_str)
        writer.writerow([user_id, recommend])

# Map50

In [68]:
val_seen_group_df["subgroup"].isna().sum()

33

In [69]:
train_user_haved_purchased_course = {}
for seen_user_id, course_ids in zip(train_df["user_id"], train_df["course_id"]):
    course_id_list = course_ids.split(' ')
    train_user_haved_purchased_course[seen_user_id] = train_user_haved_purchased_course.setdefault(seen_user_id, []) + course_id_list

val_answer_subgroups = {}
val_seen_group_df_fillna = val_seen_group_df.fillna("")
for seen_user_id, subgroup_ids in zip(val_seen_group_df_fillna["user_id"], val_seen_group_df_fillna["subgroup"]):
    if len(subgroup_ids) > 0:
        course_id_list = [ int(x) for x in subgroup_ids.split(' ')]
        val_answer_subgroups[seen_user_id] = val_answer_subgroups.setdefault(seen_user_id, []) + course_id_list
list(val_answer_subgroups.items())[:5]

[('56dae2b74e3ef90900b7bd0e', [37, 71]),
 ('60e66f29be3e3b0006c4db75', [1, 14, 15]),
 ('5c919efb728ddf00208b9b2b', [1, 3, 5, 25, 29, 50, 51, 52, 66, 67, 72, 73]),
 ('5ac115507997a2001e7c3617', [33, 34, 38, 39]),
 ('5f53b84440c5be3bb873a9d3', [30])]

In [70]:
import ml_metrics

answers = []
predictions = []
map50s = []
for user_id in val_seen_group_df["user_id"]:
    prediction = get_recommendations_new(train_user_haved_purchased_course[user_id], cosine_sim2, top = 50)

    prediction_idxs = prediction
    if user_id in val_answer_subgroups.keys():
        answer_idxs = val_answer_subgroups[user_id]
    else:
        answer_idxs = []

    # print(user_id)
    # print([course2subgroups[x] for x in train_user_haved_purchased_course[user_id]])
    # print(prediction) 
    # print(answer_idxs)
    # print("")

    predictions.append(prediction_idxs)
    answers.append(answer_idxs)
    map50s.append(ml_metrics.mapk(predicted= [prediction_idxs], actual= [answer_idxs], k = 50))
    
print(np.mean(map50s))
#answers.append(answers)        

map50 = ml_metrics.mapk(predicted= predictions, actual= answers, k = 50)
map50

0.2642995820078796


0.2642995820078796

In [71]:
0.2707003182647394

0.2707003182647394

# Use similar user

In [14]:
user_haved_purchased_course = {}

train_user_haved_purchased_course = {}
for seen_user_id, course_ids in zip(train_df["user_id"], train_df["course_id"]):
    course_id_list = course_ids.split(' ')
    train_user_haved_purchased_course[seen_user_id] = train_user_haved_purchased_course.setdefault(seen_user_id, []) + course_id_list
    user_haved_purchased_course[seen_user_id] = user_haved_purchased_course.setdefault(seen_user_id, []) + course_id_list

val_seen_user_haved_purchased_course = {}
for seen_user_id, course_ids in zip(val_seen_df["user_id"], val_seen_df["course_id"]):
    course_id_list = course_ids.split(' ')
    val_seen_user_haved_purchased_course[seen_user_id] = val_seen_user_haved_purchased_course.setdefault(seen_user_id, []) + course_id_list
    user_haved_purchased_course[seen_user_id] = user_haved_purchased_course.setdefault(seen_user_id, []) + course_id_list
    
val_unseen_user_haved_purchased_course = {}
for unseen_user_id, course_ids in zip(val_unseen_df["user_id"], val_unseen_df["course_id"]):
    course_id_list = course_ids.split(' ')
    val_unseen_user_haved_purchased_course[unseen_user_id] = val_unseen_user_haved_purchased_course.setdefault(unseen_user_id, []) + course_id_list
    user_haved_purchased_course[unseen_user_id] = user_haved_purchased_course.setdefault(unseen_user_id, []) + course_id_list

haved_purchased_users_list = []
for user_id in train_user_haved_purchased_course.keys():
    if len(train_user_haved_purchased_course[user_id]) >0:
        haved_purchased_users_list.append(user_id)
for user_id in val_seen_user_haved_purchased_course.keys():
    if len(val_seen_user_haved_purchased_course[user_id]) >0:
        haved_purchased_users_list.append(user_id)
for user_id in val_unseen_user_haved_purchased_course.keys():
    if len(val_unseen_user_haved_purchased_course[user_id]) >0:
        haved_purchased_users_list.append(user_id)
haved_purchased_users_list = list(set(haved_purchased_users_list))    

print(len(train_user_haved_purchased_course))
print(len(val_seen_user_haved_purchased_course))
print(len(val_unseen_user_haved_purchased_course))
print(len(haved_purchased_users_list))
print(len(user_haved_purchased_course))

59737
7748
11622
71359
71359


In [ ]:
f = open('subgroup_content_embed/unseen_user_similar_add_course.pickle', 'rb+')
info = pickle.load(f)
print(info)

In [79]:
with open('similar_user/unseen_user_similar_course.pickle', 'rb') as f:
    test_predict_similiar_users = pickle.load(f)
    
print(len(test_predict_similiar_users["similar_users"]))

FileNotFoundError: [Errno 2] No such file or directory: 'subgroup_content_embed/unseen_user_similar_course.pickle'

In [15]:
def get_recommend_subgroup_by_users(user_id, similar_users_list, top = 50):
    # print(user_index)
    recommend_subgroups = []
    recommend_course_ids = []
    similar_users_list #= test_predict_similiar_users[user_id]
    for similar_user in similar_users_list:
        if similar_user in val_seen_user_haved_purchased_course.keys():
            for course_id in val_seen_user_haved_purchased_course[similar_user]:
                if len(recommend_subgroups)>top:
                    break
                if course_id not in recommend_course_ids:
                    if user_id in val_unseen_user_haved_purchased_course.keys():
                        if course_id not in val_unseen_user_haved_purchased_course[user_id]:
                            recommend_course_ids.append(course_id)
                            for subgroup in course2subgroups[course_id]:
                                if subgroup not in recommend_subgroups:
                                    recommend_subgroups.append(subgroup)
                    else:
                        recommend_course_ids.append(course_id)
                        for subgroup in course2subgroups[course_id]:
                            if subgroup not in recommend_subgroups:
                                recommend_subgroups.append(subgroup)
        
        if similar_user in train_user_haved_purchased_course.keys():           
            for course_id in train_user_haved_purchased_course[similar_user]:
                if len(recommend_subgroups)>top:
                    break
                if course_id not in recommend_course_ids:
                    if user_id in val_unseen_user_haved_purchased_course.keys():
                        if course_id not in val_unseen_user_haved_purchased_course[user_id]:
                            recommend_course_ids.append(course_id)
                            for subgroup in course2subgroups[course_id]:
                                if subgroup not in recommend_subgroups:
                                    recommend_subgroups.append(subgroup)
                    else:
                        recommend_course_ids.append(course_id)
                        for subgroup in course2subgroups[course_id]:
                            if subgroup not in recommend_subgroups:
                                recommend_subgroups.append(subgroup)
                        
        if similar_user in val_unseen_user_haved_purchased_course.keys():           
            for course_id in val_unseen_user_haved_purchased_course[similar_user]:
                if len(recommend_subgroups)>top:
                    break
                if course_id not in recommend_course_ids:
                    if user_id in val_unseen_user_haved_purchased_course.keys():
                        if course_id not in val_unseen_user_haved_purchased_course[user_id]:
                            recommend_course_ids.append(course_id)
                            for subgroup in course2subgroups[course_id]:
                                if subgroup not in recommend_subgroups:
                                    recommend_subgroups.append(subgroup)
                    else:
                        recommend_course_ids.append(course_id)
                        for subgroup in course2subgroups[course_id]:
                            if subgroup not in recommend_subgroups:
                                recommend_subgroups.append(subgroup)
                
    # Return the top 10 most similar users
    return recommend_subgroups

In [16]:

test = '5f1dc5a3afdc0537b5de8979'
print(users_df.iloc[user2id_mapping[test]])  
test_predict = get_recommend_subgroup_by_users(test, test_predict_similiar_users["similar_users"][test], 50)
[ idx2subgroups[subgroup_id] for subgroup_id in test_predict ]

user_id                                       5f1dc5a3afdc0537b5de8979
gender                                                          female
occupation_titles                                              製造業,服務業
interests            投資理財_理財,程式_量化分析,投資理財_投資觀念,職場技能_效率提升,投資理財_金融商品,...
recreation_names                                             旅行旅遊,金融理財
Name: 39502, dtype: object


NameError: name 'test_predict_similiar_users' is not defined

# Predict Course by similar user and course embedding

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from scipy.sparse import csr_matrix

### seen unseen

#with open(r'similar_users\seen_user_similar_add_course.pickle', 'rb') as f:
with open(r'similar_user\unseen_user_similar_course.pickle', 'rb') as f:
    test_predict_similiar_users = pickle.load(f)
    
print(len(test_predict_similiar_users["similar_users"]))

11097


In [ ]:
def get_course_score_by_course_embedding(cosine_sim_sum, haved_courses_list, weight, cosine_sim):
    haved_courses_index_list = [ course2id_mapping[course_id] for course_id in haved_courses_list]
    for idx in haved_courses_index_list:
        # Get the pairwsie similarity scores of all courses with that course
        sim_scores = list(enumerate(cosine_sim[idx]))
        for i in range(len(sim_scores)):
            cosine_sim_sum[i][1] += weight*sim_scores[i][1]

    return cosine_sim_sum

In [ ]:
def get_recommendations_course_by_course_embedding_and_similar_users(user_id, similar_users_list, similar_user_weights_list, cosine_sim, top = 50):
    # print(user_index)
    cosine_sim_sum = [ [i, 0] for i in range(len(course2id_mapping))]

    if user_id in user_haved_purchased_course.keys():
        cosine_sim_sum = get_course_score_by_course_embedding(cosine_sim_sum, user_haved_purchased_course[user_id], 1, cosine_sim)
    for similar_user, similar_user_weight in zip(similar_users_list, similar_user_weights_list):
        cosine_sim_sum = get_course_score_by_course_embedding(cosine_sim_sum, user_haved_purchased_course[similar_user], similar_user_weight, cosine_sim)
        #print(similar_user_weight)

    sim_scores = sorted(cosine_sim_sum, key=lambda x: x[1], reverse=True)

    recommend_subgroups = []
    # Get the scores of the 50 most course not buy
    for i in range(len(sim_scores)):
        if len(recommend_subgroups) < top:
            if user_id in user_haved_purchased_course.keys():
                if id2course_mapping[sim_scores[i][0]]not in user_haved_purchased_course[user_id]:
                    course_id = course_df['course_id'][sim_scores[i][0]]
                    for subgroup in course2subgroups[course_id]:
                        if subgroup not in recommend_subgroups:
                            recommend_subgroups.append(subgroup)
            else:
                course_id = course_df['course_id'][sim_scores[i][0]]
                for subgroup in course2subgroups[course_id]:
                    if subgroup not in recommend_subgroups:
                        recommend_subgroups.append(subgroup)
        else:
            break

    # Return the top 
    return recommend_subgroups

# Predict and Save

In [ ]:
import csv
from tqdm.auto import tqdm

#test_predict_users = test_seen_df["user_id"].to_list()
test_predict_users = test_unseen_df["user_id"].to_list()
print(len(test_predict_users))
count = []
with open("predictions/unseen_subgroup_by_similar_course_embedding_75.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["user_id", "subgroup"])
    for user_id in tqdm(test_predict_users):
        #predictions = get_recommend_subgroup_by_users(user_id, test_predict_similiar_users["similar_users"][user_id], top = 50)
        predictions = get_recommendations_course_by_course_embedding_and_similar_users(user_id, 
                    test_predict_similiar_users["similar_users"][user_id][:75], test_predict_similiar_users["similar_user_weights"][user_id][:75], cosine_sim2, 50)
        recommend = " ".join([ str(x) for x in predictions])
        writer.writerow([user_id, recommend])
        count.append(len(predictions))

11097


100%|██████████| 11097/11097 [05:29<00:00, 33.69it/s]


In [ ]:
zero = 0
not_zero = [] 
for x in count:
    if x < 30:
        zero += 1
    else:
        not_zero.append(x)
print(zero)
np.mean(count)

25


47.20446967648914

In [ ]:
count[0:10]

[48, 51, 51, 48, 51, 49, 44, 51, 51, 51]